In [4]:
import numpy as np

def compute_feature_expectations(policy, feature_map, dataset):
    """Compute the feature expectations for a given policy."""
    feature_expectations = np.zeros_like(feature_map(dataset[0][0][0], dataset[0][0][1]))
    for trajectory in dataset:
        for h, a in trajectory:
            feature_expectations += feature_map(h, a)
    return feature_expectations / len(dataset)

def compute_optimal_policy(reward_weights, feature_map, dataset):
    """
    Compute the optimal policy for a given reward function.
    Placeholder for RL optimization routine.
    """
    # Replace with RL solver to compute optimal policy π for R = w · φ
    return np.random.random(len(reward_weights))  # Placeholder

def orthogonal_projection(mu_e, mu_prev, mu_current):
    """Orthogonally project μ^π_E onto the line through μ̄_k−1 and μ^π_k."""
    diff = mu_current - mu_prev
    denom = np.dot(diff, diff)
    
    if denom == 0:
        # If the difference is zero, return the previous projection
        return mu_prev
    
    projection = ((np.dot(mu_e - mu_prev, diff) / denom) * diff) + mu_prev
    return projection


def batch_max_margin_cirl(dataset, feature_map, max_iterations=100, epsilon=1e-3):
    """
    Batch, Max-Margin CIRL implementation.

    Args:
        dataset: A list of trajectories where each trajectory is a list of (h, a) tuples.
        feature_map: A function φ(h, a) to compute feature expectations.
        max_iterations: Maximum number of iterations.
        epsilon: Convergence threshold.
    
    Returns:
        R̃: Reward function weights.
        Δ: Set of feature expectations.
        Π: Set of policies.
    """
    mu_e = compute_feature_expectations(None, feature_map, dataset)  # Expert feature expectations
    w_0 = np.random.random(len(mu_e))  # Random initial reward weights
    pi_0 = compute_optimal_policy(w_0, feature_map, dataset)
    mu_pi_0 = compute_feature_expectations(pi_0, feature_map, dataset)
    
    Pi = [pi_0]
    Delta = [mu_pi_0]
    mu_bar = mu_pi_0

    for k in range(1, max_iterations + 1):
        w_k = mu_e - mu_bar
        pi_k = compute_optimal_policy(w_k, feature_map, dataset)
        mu_pi_k = compute_feature_expectations(pi_k, feature_map, dataset)

        Pi.append(pi_k)
        Delta.append(mu_pi_k)

        mu_bar = orthogonal_projection(mu_e, mu_bar, mu_pi_k)
        t = np.linalg.norm(mu_e - mu_bar, ord=2)

        if t < epsilon:
            break

    # Find the best policy in the set Π
    K = np.argmin([np.linalg.norm(mu_e - mu_pi, ord=2) for mu_pi in Delta])
    R_tilde = w_k  # Final reward function weights

    return R_tilde, Delta, Pi

# Example Usage
def feature_map(h, a):
    """Example feature map function φ(h, a)."""
    h = np.array(h)  # Ensure `h` is a numpy array
    return h * a  # Example feature map, adjust as needed.

# Example dataset (list of trajectories with (state, action) pairs)
# Ensure states (`h`) are numpy arrays and actions (`a`) are scalars
dataset = [
    [(np.array([1, 0]), 1), (np.array([0, 1]), 0)], 
    [(np.array([1, 1]), 0), (np.array([0, 0]), 1)]
]

R_tilde, Delta, Pi = batch_max_margin_cirl(dataset, feature_map)
print("Final Reward Function Weights:", R_tilde)
print("Set of Feature Expectations:", Delta)
print("Set of Policies:", Pi)


Final Reward Function Weights: [0. 0.]
Set of Feature Expectations: [array([0.5, 0. ]), array([0.5, 0. ])]
Set of Policies: [array([0.14106641, 0.54531719]), array([0.11046977, 0.70917225])]
